In [ ]:
from pydantic import BaseModel

class CoupleBase(BaseModel):
    name: str
    users: list['UserBase']

class UserBase(BaseModel):
    username: str
    firstname: str
    lastname: str
    email: str
    couple: CoupleBase
    


In [5]:
from passion_sync.db import Base
from passion_sync.model.user import Couple, User
from sqlalchemy import create_engine
from pydantic_settings import BaseSettings
from datetime import datetime

class Settings(BaseSettings):
    database_url_async: str
    test_database_url_async: str
    database_url: str
    test_database_url: str
    echo_sql: bool = True
    test: bool = True
    
settings = Settings()


engine = create_engine(settings.test_database_url, echo=settings.echo_sql)

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

2025-02-22 20:56:20,617 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-02-22 20:56:20,617 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-22 20:56:20,619 INFO sqlalchemy.engine.Engine select current_schema()
2025-02-22 20:56:20,619 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-22 20:56:20,620 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-02-22 20:56:20,621 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-22 20:56:20,623 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-22 20:56:20,625 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [4]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(engine)

def create_user(name):
    with Session.begin() as session:
        user = User(
            username=name
        )
        session.add(user)
    return  user

def create_couple(name:str, users: list[User]=[]):
    with Session.begin() as session:
        couple = Couple(name=name, users=users)
        session.add_all([couple] + users)
    return couple

def add_activity(user: User, activit_type: ActivityType, value: str):
    with Session.begin() as session:
        activity = Activity(type=activit_type, value=value, created=datetime.now())
        user = session.merge(user)
        user.activities.append(activity)
        session.add(activity)

In [ ]:
user1 = create_user("test_user1")
user2 = create_user("test_user2")
couple = create_couple("test_couple",users=[user1, user2])

add_activity(user1, ActivityType.GENERIC, "test_activity")



In [6]:
from sqlalchemy import select

with Session() as session:
    stmt = select(Couple)
    couple = session.scalar(stmt)
    print(couple.name)
    print([[u.username, [a.value for a in u.activities]] for u in couple.users])

test_couple
[['test_user1', ['test_activity']], ['test_user2', []]]
